**Provenance du code :** https://github.com/pmongruel/StatApp/blob/main/Data%20Process/database_creation.ipynb

This Notebook:
- merge 'database_votants_herve.csv' and 'database_scrutins_herve.csv' to create  'database_votes_herve.csv'
- create dataset with socio-demographic data on each constituency

## Voting data

In [1]:
import pandas as pd
import re
import os

In [2]:
path = 'C:/Users/veve1/OneDrive/Documents/ENSAE 3A/S2/Social Choice and Voting/Project/Social-Choice-and-Voting'
data_path = os.path.join(path,'Data')

In [3]:
df1 = pd.read_csv(os.path.join(data_path,'database_votants_herve.csv'))
df2 = pd.read_csv(os.path.join(data_path,'database_scrutins_herve.csv'))

In [4]:
df_votes = df1.merge(df2, on="idScrutin", how='right')

In [5]:
df_votes = df_votes[['idScrutin', 'idVotant', 'vote', 'organeRefGroupe', 'date_scrutin', 'code_type_vote',
                    'titre', 'demandeur', 'organe_ref', 'votants', 'pour', 'contre', 'non_votants', 
                     'non_votants_volontaires', 'abstention', 'resultat']]
df_votes

,idScrutin,idVotant,vote,organeRefGroupe,date_scrutin,code_type_vote,titre,demandeur,organe_ref,votants,pour,contre,non_votants,non_votants_volontaires,abstention,resultat
0,0,PA606171,Non-votant,PO730964,2020-10-07,SPO,l'article 24 bis du projet de loi portant dive...,"Président du groupe ""La République en Marche""",PO717460,57,56,0,2,0,1,adopté
1,0,PA721824,Non-votant,PO730964,2020-10-07,SPO,l'article 24 bis du projet de loi portant dive...,"Président du groupe ""La République en Marche""",PO717460,57,56,0,2,0,1,adopté
2,0,PA719952,Pour,PO730964,2020-10-07,SPO,l'article 24 bis du projet de loi portant dive...,"Président du groupe ""La République en Marche""",PO717460,57,56,0,2,0,1,adopté
3,0,PA607395,Pour,PO730964,2020-10-07,SPO,l'article 24 bis du projet de loi portant dive...,"Président du groupe ""La République en Marche""",PO717460,57,56,0,2,0,1,adopté
4,0,PA718710,Pour,PO730964,2020-10-07,SPO,l'article 24 bis du projet de loi portant dive...,"Président du groupe ""La République en Marche""",PO717460,57,56,0,2,0,1,adopté
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
450096,4416,PA335612,Pour,PO730940,2018-12-04,SPO,l'amendement de suppression n°54 de Mme Geneva...,"Président du groupe ""Les Républicains""Présiden...",PO717460,158,64,93,2,0,1,rejeté
450097,4416,PA607619,Contre,PO759900,2018-12-04,SPO,l'amendement de suppression n°54 de Mme Geneva...,"Président du groupe ""Les Républicains""Présiden...",PO717460,158,64,93,2,0,1,rejeté
450098,4416,PA719608,Pour,PO723569,2018-12-04,SPO,l'amendement de suppression n°54 de Mme Geneva...,"Président du groupe ""Les Républicains""Présiden...",PO717460,158,64,93,2,0,1,rejeté
450099,4416,PA721932,Pour,PO723569,2018-12-04,SPO,l'amendement de suppression n°54 de Mme Geneva...,"Président du groupe ""Les Républicains""Présiden...",PO717460,158,64,93,2,0,1,rejeté


In [6]:
df_votes.to_csv(os.path.join(data_path,'database_votes_herve.csv'))

## Constituency data

In [7]:
df_circo = pd.read_csv(os.path.join(data_path,'Copie de Data circo legislatives 2022.csv'),encoding = "utf-8",sep = ";")

In [8]:
df_circo.drop(labels = [0,1,2,3,4,5,7],axis = 0, inplace = True)
df_circo.rename(columns=df_circo.iloc[0],inplace=True)
df_circo.drop(df_circo.index[0],inplace = True)

In [9]:
df_circo = df_circo[['circo','Nom de la circonscription','pop_urb','pop_rur_periu','pop_rur_non_periu', 'modtrans_aucun',
                         'modtrans_pied','modtrans_velo','modtrans_moto','modtrans_voit','modtrans_commun']]

df_circo.replace(',', '.', regex=True, inplace = True)
columns = list(df_circo.columns)
columns.remove('circo')
columns.remove('Nom de la circonscription')
for column in columns :
    df_circo[column] = df_circo[column].apply(lambda x : pd.to_numeric(x, errors = 'coerce'))
    
df_circo['urbain'] = df_circo[['pop_urb']]
df_circo['non-urbain'] = df_circo['pop_rur_periu'] + df_circo['pop_rur_non_periu']
df_circo['median_voter_is_urban'] = df_circo['urbain'].apply(lambda x : x >= 50)

# Mode de transports
df_circo['moto/voiture'] = df_circo[['modtrans_moto','modtrans_voit']].sum(axis=1)
df_circo['pied/velo/transport en commun'] = df_circo[['modtrans_aucun','modtrans_pied','modtrans_velo','modtrans_commun']].sum(axis = 1)
df_circo['median_voter_isnot_car_driver'] = df_circo['pied/velo/transport en commun'].apply(lambda x : x >= 50)


In [10]:
df_circo[['Département', 'Numéro']] = df_circo['Nom de la circonscription'].str.split(' - ', n=1, expand=True)
df_circo['Numéro de circonscription'] = df_circo['Numéro'].apply(lambda x: re.findall(r'\d+', x)[0] if re.findall(r'\d+', x) else 1)
df_circo['Numéro de circonscription'] = df_circo['Numéro de circonscription'].astype(int)

In [11]:
df_circo['Département'] = df_circo['Département'].str.strip()
df_circo['Département'].replace({'Alpes de Haute-Provence' : 'Alpes-de-Haute-Provence',
                                 'La Réunion' : 'Réunion'
                                }, inplace = True)

In [12]:
df_circo.to_csv(os.path.join(data_path,'database_circonscriptions.csv')) 